In [ ]:
import os
os.environ['WANDB_DISABLED']="true"

# Install and load libraries

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib rouge_score evaluate
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 34.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of

In [ ]:
# install git-fls for pushing model and logs to the hugging face hub
!sudo apt-get install git-lfs --yes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!nvidia-smi -L

GPU 0: NVIDIA L4 (UUID: GPU-2612e56f-1696-5891-e5cc-8493a5ea2884)


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import os
import torch

from datasets import Dataset, load_metric

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# set up the model with Lora

Code reference for setting up Lora: https://colab.research.google.com/drive/14xo6sj4dARk8lXZbOifHEn1f_70qNAwy?usp=sharing#scrollTo=4iwHGzKBN6wk

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb

model_name = "allenai/led-large-16384"
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    load_in_8bit=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="Seq2Seq" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1572864 || all params: 461374464 || trainable%: 0.34090833427660183


In [ ]:
!nvidia-smi

Mon Jul 15 22:30:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   54C    P0              29W /  72W |    751MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Load data

In [ ]:
# Paths to the files
path_train_long = '/content/drive/MyDrive/W266_Project/Data/long_dialogue_NER_cleaning/train_long_clinical_NER.csv'
path_test_long = '/content/drive/MyDrive/W266_Project/Data/long_dialogue_NER_cleaning/test_long_clinical_NER.csv'
path_val_long = '/content/drive/MyDrive/W266_Project/Data/long_dialogue_NER_cleaning/val_long_clinical_NER.csv'

# Load the datasets
train_long = pd.read_csv(path_train_long)
test_long = pd.read_csv(path_test_long)
val_long = pd.read_csv(path_val_long)

# Check the first few rows to ensure they're loaded correctly
print("long_dialogue_train_data:")
print(train_long.head())

# Check the shape
print("Shape of long_dialogue_train_data:")
print(train_long.shape)
print("Shape of long_dialogue_test_data:")
print(test_long.shape)
print("Shape of long_dialogue_val_data:")
print(val_long.shape)


long_dialogue_train_data:
                                            dialogue  \
0  doctor donna torres , date of birth , 08/01/19...   
1  doctor: Good morning, Mr. patient. I'm Dr. doc...   
2  doctor: Hello Mrs. patient, thank you for comi...   
3  doctor hi virginia how're you today patient i'...   
4  doctor: Hello, Mrs. patient, welcome back. How...   

                                                note  \
0  SUBJECTIVE CHIEF COMPLAINT Annual health maint...   
1  SUBJECTIVE CHIEF COMPLAINT Patient reports fru...   
2  SUBJECTIVE CHIEF COMPLAINT Left arm pain after...   
3  SUBJECTIVE CHIEF COMPLAINT Right knee pain. HI...   
4  SUBJECTIVE CHIEF COMPLAINT Recurrent low back ...   

                                  clinical_ner_label  \
0  [{'phrase': 'fibro aden omas', 'label': 'DISEA...   
1  [{'phrase': 'treated', 'label': 'HISTORY'}, {'...   
2  [{'phrase': 'lipid', 'label': 'DISEASE_DISORDE...   
3  [{'phrase': 'days', 'label': 'DURATION'}, {'ph...   
4  [{'phrase': 'bloo

In [ ]:
train_long.head(1)

,dialogue,note,clinical_ner_label,clinical_ner_unique_label,clinical_ner_no_label,clinical_ner_unique_no_label
0,"doctor donna torres , date of birth , 08/01/19...",SUBJECTIVE CHIEF COMPLAINT Annual health maint...,"[{'phrase': 'fibro aden omas', 'label': 'DISEA...","[{'phrase': 'fibro aden omas', 'label': 'DISEA...","['fibro aden omas', '40', '45', 'breast', 'can...","['fibro aden omas', '40', '45', 'breast', 'can..."


In [ ]:
def convert_to_string(list_str):
    # Convert string representation of list to an actual list
    actual_list = eval(list_str)
    # Join the list elements into a single string separated by ", "
    return ", ".join(actual_list)

# Apply the function to convert each list to a string
for df in [train_long, test_long, val_long]:
    df['clinical_ner_no_label'] = df['clinical_ner_no_label'].apply(convert_to_string)
    df['clinical_ner_unique_no_label'] = df['clinical_ner_unique_no_label'].apply(convert_to_string)


In [ ]:
train_long.head(1)

,dialogue,note,clinical_ner_label,clinical_ner_unique_label,clinical_ner_no_label,clinical_ner_unique_no_label
0,"doctor donna torres , date of birth , 08/01/19...",SUBJECTIVE CHIEF COMPLAINT Annual health maint...,"[{'phrase': 'fibro aden omas', 'label': 'DISEA...","[{'phrase': 'fibro aden omas', 'label': 'DISEA...","fibro aden omas, 40, 45, breast, cancer, cance...","fibro aden omas, 40, 45, breast, cancer, ultra..."


The model can take 16384 tokens, based on data analysis:
- Min dialogue length: 135
- Ave dialogue length: 426
- Max dialogue length: 3050
- 95% dialogue length: 966
- median dialogue length: 353
- Min summary length: 83
- Ave summary length: 276
- Max summary length: 752
- 95% summary length: 458
- Median summary length: 257

In [ ]:
train_long = Dataset.from_pandas(train_long)
test_long = Dataset.from_pandas(test_long)
val_long = Dataset.from_pandas(val_long)


# Fine-tuning the model

## Tokenization

Code reference for tokenization: https://colab.research.google.com/drive/12LjJazBl7Gam0XBPy_y0CTOJZeZ34c2v?usp=sharing

In [ ]:
# tokennize
max_input_length = 2048
max_output_length = 1024
def process_data_to_model_inputs(batch):
    start_prompt = (
    "Summarize the following patient-doctor dialogue into 2 sections: "
    "\"SUBJECTIVE CHIEF COMPLAINT\" and \"ASSESSMENT AND PLAN\". "
    "Include all medically relevant information, including family history, "
    "diagnosis, past medical and surgical history, immunizations, lab results, and known allergies. "
    "Use the following clinical entities extracted from the dialogue to help summarization, but do not overly use them. "
    "Entities:\n\n")

    # Define the end of the prompt to indicate where the summary should go
    end_prompt = "\n\nSummary: "

    # Construct the full prompt for each dialogue in the batch, using the corresponding entities
    prompt = [start_prompt + entities + 'Dialogue: \n\n' + dialogue + end_prompt
               for entities, dialogue in zip(batch['clinical_ner_unique_no_label'], batch['dialogue'])]

    # tokenize the inputs and labels
    inputs = tokenizer(
        prompt,
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["note"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [ ]:
# map the function
batch_size = 2
train_long = train_long.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=[ "note", "dialogue","clinical_ner_label","clinical_ner_unique_label","clinical_ner_unique_label","clinical_ner_unique_no_label"],
)

Map:   0%|          | 0/1102 [00:00<?, ? examples/s]

In [ ]:
test_long = test_long.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=[ "note", "dialogue","clinical_ner_label","clinical_ner_unique_label","clinical_ner_unique_label","clinical_ner_unique_no_label"],
)

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

In [ ]:
val_long = val_long.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=[ "note", "dialogue","clinical_ner_label","clinical_ner_unique_label","clinical_ner_unique_label","clinical_ner_unique_no_label"],
)

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

In [ ]:
from random import sample
# check the shape after tokenization
sample_input_id = train_long['input_ids'][0:1]
sample_label = train_long['labels'][0:1]
sample_attention_mask = train_long['attention_mask'][0:1]
sample_global_attention_mask = train_long['global_attention_mask'][0:1]

print("tokenized_train_exp: ", train_long)
print("\nsample_input_id: ", len(sample_input_id[0]), sample_input_id)
print("\nsample_label: ", len(sample_label[0]), sample_label)
print("\nsample_attention_mask: ", len(sample_attention_mask[0]), sample_attention_mask)
print("\nsample_global_attention_mask: ", len(sample_global_attention_mask[0]), sample_global_attention_mask)


tokenized_train_exp:  Dataset({
    features: ['clinical_ner_no_label', 'input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
    num_rows: 1102
})

sample_input_id:  2048 [[0, 38182, 3916, 2072, 5, 511, 3186, 12, 44914, 6054, 88, 132, 9042, 35, 22, 104, 12027, 33302, 10002, 3858, 7720, 597, 15823, 43362, 17831, 113, 8, 22, 17042, 12147, 12613, 4248, 37330, 845, 39682, 70, 25749, 4249, 335, 6, 217, 284, 750, 6, 9726, 6, 375, 1131, 8, 15535, 750, 6, 13998, 18391, 6, 6348, 775, 6, 8, 684, 26331, 4, 7627, 5, 511, 5154, 8866, 27380, 31, 5, 6054, 7, 244, 39186, 1938, 6, 53, 109, 45, 14860, 304, 106, 4, 9860, 2192, 35, 50118, 50118, 506, 1452, 1001, 2329, 225, 1021, 13738, 6, 843, 6, 2248, 6, 6181, 6, 1668, 6, 29976, 6, 265, 360, 48201, 35, 1437, 50118, 50118, 44914, 218, 2133, 16535, 1535, 2156, 1248, 9, 3113, 2156, 12112, 73, 2663, 73, 34499, 479, 3299, 20280, 218, 2133, 27785, 141, 32, 47, 17487, 3186, 939, 437, 205, 479, 141, 59, 47, 17487, 3299, 939, 437, 608, 157, 2156, 33

In [ ]:
decoded_text = tokenizer.decode(sample_input_id[0], skip_special_tokens=True)
decoded_text

'Summarize the following patient-doctor dialogue into 2 sections: "SUBJECTIVE CHIEF COMPLAINT" and "ASSESSMENT AND PLAN". Include all medically relevant information, including family history, diagnosis, past medical and surgical history, immunizations, lab results, and known allergies. Use the following clinical entities extracted from the dialogue to help summarization, but do not overly use them. Entities:\n\nfibro aden omas, 40, 45, breast, cancer, ultrasound, business daysDialogue: \n\ndoctor donna torres, date of birth, 08/01/1980. doctor hi donna! how are you? patient i\'m good. how about you? doctor i\'m doing well, thank you. and so, i saw that dr. brown put you on buspar. have you been on that before? patient no, that\'s new. doctor okay. how is it working for you? patient my anxiety is going good now, thankfully. i\'m serious, it was brutal in november and december. finally, i was like, " i can not do this. " i have no idea why it happened. dr. ward did put me on singulair, a

## Training

In [ ]:

import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from transformers import DataCollatorForSeq2Seq

# ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.optim import AdamW

# Define custom optimizer
optimizer = AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)

# Define the output directory
#repository_id = "./output_directory"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir="outputs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=6,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=3e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    #logging & evaluation strategies
    #logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_long,
    eval_dataset=val_long,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None),  # Pass the custom optimizer
)

In [ ]:
# Start training
with torch.cuda.amp.autocast():
    trainer.train()


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.650289,0.014600,0.000000,0.007300,0.014600,20.000000
2,No log,1.382697,0.723400,0.371900,0.706800,0.729500,20.000000
3,No log,1.267849,0.509200,0.236100,0.486800,0.499000,20.000000
4,1.671200,1.231273,0.541600,0.274800,0.546200,0.543600,20.000000
5,1.671200,1.224720,0.454200,0.243400,0.444200,0.448100,20.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1253: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1253: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 

In [ ]:
!nvidia-smi

Tue Jul 16 01:28:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   74C    P0              34W /  72W |  10941MiB / 23034MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# save the model
trainer.save_model("./led_large_dialogue_G_ACI_clinical_unique_ner")
tokenizer.save_pretrained("./led_large_dialogue_G_ACI_clinical_unique_ner")

('./led_large_dialogue_G_ACI_clinical_unique_ner/tokenizer_config.json',
 './led_large_dialogue_G_ACI_clinical_unique_ner/special_tokens_map.json',
 './led_large_dialogue_G_ACI_clinical_unique_ner/vocab.json',
 './led_large_dialogue_G_ACI_clinical_unique_ner/merges.txt',
 './led_large_dialogue_G_ACI_clinical_unique_ner/added_tokens.json',
 './led_large_dialogue_G_ACI_clinical_unique_ner/tokenizer.json')

In [ ]:
# push to huggingface
from huggingface_hub import HfApi, HfFolder

model_name = "litt5/led_large_dialogue_G_ACI_clinical_unique_ner"
api = HfApi()

# Create the repository if it doesn't exist
api.create_repo(repo_id=model_name, repo_type="model", exist_ok=True)

# Upload the model files
api.upload_folder(
    folder_path="./led_large_dialogue_G_ACI_clinical_unique_ner",
    repo_id=model_name,
    repo_type="model"
)

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/litt5/led_large_dialogue_G_ACI_clinical_unique_ner/commit/7bc31a5abd95530785860f084201d4fe52b15512', commit_message='Upload folder using huggingface_hub', commit_description='', oid='7bc31a5abd95530785860f084201d4fe52b15512', pr_url=None, pr_revision=None, pr_num=None)

# Model evaluation

## Load the model
Code reference for quntination: https://github.com/datasci-w266/2024-summer-main/blob/master/materials/lesson_notebooks/lesson_9_Entities_and_Linking.ipynb

In [ ]:
!pip install -q accelerate
!pip install -q bitsandbytes

In [ ]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_name = "litt5/led_large_dialogue_G_ACI_clinical_unique_ner"

In [ ]:
from transformers import BitsAndBytesConfig


nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, quantization_config=nf4_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

## load and prepare the data

In [ ]:
# Paths to the filespath_train_long
path_test_long = '/content/drive/MyDrive/W266_Project/Data/long_dialogue_NER_cleaning/test_long_clinical_NER.csv'

# Load the datasets

test_long = pd.read_csv(path_test_long)


# Check the shape

print("Shape of long_dialogue_test_data:")
print(test_long.shape)



Shape of long_dialogue_test_data:
(180, 6)


In [ ]:
test_long.head(1)

,dialogue,note,clinical_ner_label,clinical_ner_unique_label,clinical_ner_no_label,clinical_ner_unique_no_label
0,"doctor: Good morning, Mr. A. Thank you for com...",SUBJECTIVE CHIEF COMPLAINT New patient evaluat...,"[{'phrase': 'July of 2006', 'label': 'DATE'}, ...","[{'phrase': 'July of 2006', 'label': 'DATE'}, ...","['July of 2006', 'August of 2007', 'seizures',...","['July of 2006', 'August of 2007', 'seizures',..."


In [ ]:
def convert_to_string(list_str):
    # Convert string representation of list to an actual list
    actual_list = eval(list_str)
    # Join the list elements into a single string separated by ", "
    return ", ".join(actual_list)

# Apply the function to convert each list to a string
test_long['clinical_ner_no_label'] = test_long['clinical_ner_no_label'].apply(convert_to_string)
test_long['clinical_ner_unique_no_label'] = test_long['clinical_ner_unique_no_label'].apply(convert_to_string)

In [ ]:
test_long.head(1)

,dialogue,note,clinical_ner_label,clinical_ner_unique_label,clinical_ner_no_label,clinical_ner_unique_no_label
0,"doctor: Good morning, Mr. A. Thank you for com...",SUBJECTIVE CHIEF COMPLAINT New patient evaluat...,"[{'phrase': 'July of 2006', 'label': 'DATE'}, ...","[{'phrase': 'July of 2006', 'label': 'DATE'}, ...","July of 2006, August of 2007, seizures, hemi p...","July of 2006, August of 2007, seizures, hemi p..."


In [ ]:
num_chunks = 10

# Split the DataFrame
dfs = np.array_split(test_long, num_chunks)

In [ ]:
# save dialogue and note to list for future Rough score
dialogue_lists = [subset["dialogue"].tolist() for subset in dfs]
note_lists = [subset["note"].tolist() for subset in dfs]

In [ ]:
datasets = []
for i in range(len(dfs)):
    dataset = Dataset.from_pandas(dfs[i])
    datasets.append(dataset)

In [ ]:
datasets

[Dataset({
     features: ['dialogue', 'note', 'clinical_ner_label', 'clinical_ner_unique_label', 'clinical_ner_no_label', 'clinical_ner_unique_no_label'],
     num_rows: 18
 }),
 Dataset({
     features: ['dialogue', 'note', 'clinical_ner_label', 'clinical_ner_unique_label', 'clinical_ner_no_label', 'clinical_ner_unique_no_label'],
     num_rows: 18
 }),
 Dataset({
     features: ['dialogue', 'note', 'clinical_ner_label', 'clinical_ner_unique_label', 'clinical_ner_no_label', 'clinical_ner_unique_no_label'],
     num_rows: 18
 }),
 Dataset({
     features: ['dialogue', 'note', 'clinical_ner_label', 'clinical_ner_unique_label', 'clinical_ner_no_label', 'clinical_ner_unique_no_label'],
     num_rows: 18
 }),
 Dataset({
     features: ['dialogue', 'note', 'clinical_ner_label', 'clinical_ner_unique_label', 'clinical_ner_no_label', 'clinical_ner_unique_no_label'],
     num_rows: 18
 }),
 Dataset({
     features: ['dialogue', 'note', 'clinical_ner_label', 'clinical_ner_unique_label', 'clinic

## Tokonization

In [ ]:
# tokennize
max_input_length = 2048
max_output_length = 1024
def process_data_to_model_inputs(batch):
    start_prompt = (
    "Summarize the following patient-doctor dialogue into 2 sections: "
    "\"SUBJECTIVE CHIEF COMPLAINT\" and \"ASSESSMENT AND PLAN\". "
    "Include all medically relevant information, including family history, "
    "diagnosis, past medical and surgical history, immunizations, lab results, and known allergies. "
    "Use the following clinical entities extracted from the dialogue to help summarization, but do not overly use them. "
    "Entities:\n\n")

    # Define the end of the prompt to indicate where the summary should go
    end_prompt = "\n\nSummary: "

    # Construct the full prompt for each dialogue in the batch, using the corresponding entities
    prompt = [start_prompt + entities + 'Dialogue: \n\n' + dialogue + end_prompt
               for entities, dialogue in zip(batch['clinical_ner_unique_no_label'], batch['dialogue'])]

    # tokenize the inputs and labels
    inputs = tokenizer(
        prompt,
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["note"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids


    return batch

In [ ]:
batch_size = 2
for i in range(6):
    processed_dataset = datasets[i].map(
        process_data_to_model_inputs,
        batched=True,
        batch_size=batch_size,
        remove_columns=["note", "dialogue","clinical_ner_label","clinical_ner_unique_label","clinical_ner_unique_label","clinical_ner_unique_no_label"],
    )
    # Create a variable name dynamically
    globals()[f'dataset_{i}'] = processed_dataset

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

In [ ]:
dataset_0

Dataset({
    features: ['clinical_ner_no_label', 'input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
    num_rows: 18
})

In [ ]:
## check the tokenization
sample_input_id = dataset_0['input_ids'][0:1]


In [ ]:
# check the prompt:
decoded_text = tokenizer.decode(sample_input_id[0], skip_special_tokens=True)
decoded_text

'Summarize the following patient-doctor dialogue into 2 sections: "SUBJECTIVE CHIEF COMPLAINT" and "ASSESSMENT AND PLAN". Include all medically relevant information, including family history, diagnosis, past medical and surgical history, immunizations, lab results, and known allergies. Use the following clinical entities extracted from the dialogue to help summarization, but do not overly use them. Entities:\n\nJuly of 2006, August of 2007, seizures, hemi par esis, hospital, C. diff, nursing home, Kep pra, Deca dron, oxycodone, aspirin, Pri lose c, Dil ant in, Flag yl, imaging studies, malignant, men ing i oma, progression, disease, significant, brain, edema, tumor, swelling, confusion, ing i oma, Chemotherapy, care, specialistDialogue: \n\ndoctor: Good morning, Mr. A. Thank you for coming in today. I understand you\'re here for a new patient evaluation. patient: That\'s right. doctor: Can you tell me a little bit about your medical history? patient: Well, I\'ve had two surgeries in th

## Prediction


In [ ]:
def model_generation(model, tokenizer, dataset):
    predictions = []

    for example in dataset:
        input_ids = example["input_ids"]
        attention_mask = example["attention_mask"]
        global_attention_mask = example["global_attention_mask"]
        reference_ids = example["labels"]

        # Decode input_ids
        input_text = tokenizer.decode(input_ids, skip_special_tokens=True)

        # Generate summary
        outputs = model.generate(input_ids=torch.tensor([input_ids]).to(device),
                                 attention_mask=torch.tensor([attention_mask]).to(device),
                                 global_attention_mask=torch.tensor([global_attention_mask]).to(device),
                                 num_beams= 4,
                                 min_length=100,
                                 max_length=512,
                                 length_penalty=2,
                                 no_repeat_ngram_size=3)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Append to lists for ROUGE calculation
        predictions.append(generated_text)
    return predictions

In [ ]:
with torch.cuda.amp.autocast():
  predictions_sub_0 = model_generation(model, tokenizer, dataset_0)

In [ ]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores for each pair of prediction and reference
rouge_scores = []
for pred, ref in zip(predictions_sub_0, note_lists[0]):
    score = scorer.score(pred, ref)
    rouge_scores.append(score)


# Calculate average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

# Print average ROUGE scores
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

Average ROUGE-1: 0.5388207108955689
Average ROUGE-2: 0.285907165011699
Average ROUGE-L: 0.29591570526276734


In [ ]:
data = {
    "Dialogue": dialogue_lists[0],
    "Reference": note_lists[0],
    "Prediction": predictions_sub_0,
}
df = pd.DataFrame(data)

import io
import os

# Specify the directory to save the CSV files
output_dir = '/content/drive/MyDrive/W266_Project/Data/Results'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)
df.to_csv(os.path.join(output_dir,"led_large_lora_clinical_unique_predictions_sub_0.csv"), index=False)

In [ ]:
with torch.cuda.amp.autocast():
  predictions_sub_1 = model_generation(model, tokenizer, dataset_1)


In [ ]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores for each pair of prediction and reference
rouge_scores = []
for pred, ref in zip(predictions_sub_1, note_lists[1]):
    score = scorer.score(pred, ref)
    rouge_scores.append(score)


# Calculate average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

# Print average ROUGE scores
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

Average ROUGE-1: 0.5411730026006465
Average ROUGE-2: 0.3002209830321238
Average ROUGE-L: 0.2913417812750139


In [ ]:
data = {
    "Dialogue": dialogue_lists[1],
    "Reference": note_lists[1],
    "Prediction": predictions_sub_1,
}
df = pd.DataFrame(data)

import io
import os

# Specify the directory to save the CSV files
output_dir = '/content/drive/MyDrive/W266_Project/Data/Results'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)
df.to_csv(os.path.join(output_dir,"led_large_lora_clinical_unique_predictions_sub_1.csv"), index=False)